# Ghost Architect — Colab T4 Main Notebook

This is the single notebook to run full **Gemma-3 Trinity training** on Google Colab T4 (16GB).

## What this notebook does
1. Validates T4 runtime
2. Installs exact dependencies
3. Syncs project files
4. Writes full T4 training configuration
5. Launches training
6. Exports GGUF artifacts


## 1) Runtime Check (must be T4 GPU)

In [1]:
import torch

assert torch.cuda.is_available(), 'CUDA is not available. Set Runtime > GPU in Colab.'
gpu_name = torch.cuda.get_device_name(0)
gpu_mem_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3
print(f'GPU: {gpu_name}')
print(f'VRAM: {gpu_mem_gb:.1f} GB')
if 'T4' not in gpu_name:
    print('Warning: This notebook is tuned for T4; adjust config if using a different GPU.')

!nvidia-smi


GPU: Tesla T4
VRAM: 14.6 GB
Wed Feb 25 08:16:16 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P8             11W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-------------------

## 2) Install Dependencies

In [2]:
!pip install -q --retries 10 --timeout 120 --upgrade pip setuptools wheel "jedi>=0.16"
!pip install -q --retries 10 --timeout 120 "unsloth==2026.1.4"
!pip install -q --retries 10 --timeout 120 "trl>=0.18.2,<=0.24.0,!=0.19.0"
!pip install -q --retries 10 --timeout 120 peft accelerate bitsandbytes datasets numpy scipy tqdm
!pip install -q --retries 10 --timeout 120 "torch>=2.1.0" "transformers>=4.38.0"
!pip check || true

print('Dependencies installed. xformers is optional and intentionally not force-installed.')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 35.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 99.8 MB/s eta 0:00:00
No broken requirements found.
Dependencies installed. xformers is optional and intentionally not force-installed.


## 3) Mount Drive and Sync Repository

In [3]:
!find /content/drive/MyDrive -maxdepth 6 -type f -path "*/src/train_vision.py"

find: ‘/content/drive/MyDrive’: No such file or directory


In [4]:
# 3) Mount Drive and Enter Project Folder
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to your project in Drive
# NOTE: Ensure you uploaded the folder named 'ghost_architect_gemma3' to your Drive root
PROJECT_DIR = '/content/drive/MyDrive/ghost_architect_gemma3'

# Check if the folder exists before trying to enter it
if not os.path.exists(PROJECT_DIR):
    print(f"❌ Error: Could not find project at {PROJECT_DIR}")
    print("Please make sure you dragged the 'ghost_architect_gemma3' folder to the main 'My Drive' screen.")
    print("Listing folders in your Drive to help debug:")
    !ls /content/drive/MyDrive | head -n 10
else:
    # Change directory directly INTO the Drive folder
    %cd {PROJECT_DIR}
    
    # Verify we are in the right place
    print(f"✅ Success! Current working directory: {os.getcwd()}")
    print("\nFolder contents:")
    !ls
    
    # Verify the critical Vision Trainer exists
    if os.path.exists('src/train_vision.py'):
        print("\n✅ 'src/train_vision.py' found. Ready for training!")
    else:
        print("\n⚠️ Warning: 'src/train_vision.py' not found. Did you upload the 'src' folder?")

Mounted at /content/drive
/content/drive/MyDrive/ghost_architect_gemma3
✅ Success! Current working directory: /content/drive/MyDrive/ghost_architect_gemma3

Folder contents:
configs  data  src

✅ 'src/train_vision.py' found. Ready for training!


## 3.5) Environment Validation

In [5]:
!python scripts/validate_environment.py

python3: can't open file '/content/drive/MyDrive/ghost_architect_gemma3/scripts/validate_environment.py': [Errno 2] No such file or directory


## 4) Full T4 Trinity Training Config

In [6]:
config_yaml = '''
model_name: "unsloth/gemma-3-12b-it-bnb-4bit"
max_seq_length: 4096
load_in_4bit: true

lora:
  r: 64
  lora_alpha: 32
  target_modules:
    - q_proj
    - k_proj
    - v_proj
    - o_proj
    - gate_proj
    - up_proj
    - down_proj
  use_rslora: true
  use_dora: true
  lora_dropout: 0.1

training:
  per_device_train_batch_size: 1
  gradient_accumulation_steps: 4
  learning_rate: 2e-4
  max_steps: 60
  warmup_steps: 10
  logging_steps: 1
  save_steps: 20
  optimizer: "adamw_8bit"
  lr_scheduler_type: "cosine"

output:
  adapters_dir: "output/adapters"
  checkpoints_dir: "output/checkpoints"
  gguf_dir: "output/gguf"

oom_fallbacks:
  - {action: reduce_seq_len, value: 2048}
  - {action: reduce_rank, value: 32}
  - {action: disable_dora, value: false}
'''

os.makedirs('configs', exist_ok=True)
with open('configs/training_config_colab_t4.yaml', 'w') as f:
    f.write(config_yaml)

print('Wrote configs/training_config_colab_t4.yaml')

Wrote configs/training_config_colab_t4.yaml


## 5) Validate Dataset

In [7]:
import os, json

dataset_path = 'data/dataset.json'
assert os.path.exists(dataset_path), f'Missing dataset: {dataset_path}'

with open(dataset_path, 'r') as f:
    raw = f.read().strip()

if raw:
    _ = json.loads(raw)
    print('Dataset JSON is valid.')
else:
    print('Dataset file is empty. Populate data/dataset.json before training.')

Dataset JSON is valid.


## 6) Launch Full Training

In [8]:
!python src/train_vision.py --dataset data/dataset.json

2026-02-25 08:25:39.939868: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1772007939.960673    3767 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1772007939.967457    3767 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1772007939.985208    3767 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1772007939.985231    3767 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1772007939.985235    3767 computation_placer.cc:177] computation placer alr

## 7) Export to GGUF

In [ ]:
# Expected CLI in future implementation:
# !python src/export.py --adapter_dir output/adapters --output_dir output/gguf --quantization q4_k_m

print('Implement src/export.py next, then run the command above.')